# Maastikuökoloogia

__Evelyn Uuemaa, PhD__
__Alexander Kmoch, PhD__

- Praktikumi lisa

Tudengid laevad üles oma ZIP-faili, mis sisaldab maastike rasterfaile (geotifid). Rasterfailidele arvutatakse maastikuindeksid. Tulemuse saab tabelina alla laadida.

Kasutatud materjalid:
- https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0225734
- https://github.com/martibosch/pylandstats

## Kuidas seda skripti kasutada

Mine hiirega kõige esimesele kastile ja vajuta "Run" nuppu, et üksus läbi jooksutada ja minna järgmiste plokkide juurde. Kui midagi valesti läheb, saad sa lihtsasti uuesti algusest alustada.

In [1]:
from IPython.display import display
import ipywidgets as widgets
import zipfile
import glob
from tqdm import tqdm 

Ülejärgmise ploki juures tuleb sul arvutist valida ZIP-fail, mis sisaldab maastike rasterfaile.

In [2]:
upload = widgets.FileUpload( accept='.zip', multiple=False )

In [3]:
upload

FileUpload(value={}, accept='.zip', description='Upload')

Kui oled faili oma arvutist valinud, peaks üleslaadimise lahter näitama "Upload (1)", mis tähendab, et fail on nüüd valitud.

Jooksuta läbi järgmine plokk. 

In [7]:
t = upload.data[0]

with open('tmp.zip', 'wb') as fh:
    fh.write(t)


with zipfile.ZipFile('tmp.zip', 'r') as zip_ref:
    zip_ref.extractall(".")

all_tifs = glob.glob("*.tif")

# uncomment to see the names of all *.tif files inside the uploaded zip 
# print(all_tifs)

Nüüd on andmed üles laetud ja saame arvutada igale maastikule maastikuindeksid: servatihedus (edge density), eraldiste tihedus (patch density), Shannoni mitmekesisuse indeks (Shannon's diversity metric), kuju indeks (shape index) ja keskmine eraldise suurus (mean patch size).
Tulemused saad allalaadida csv-failina, mille nimi on landscape_metrics.csv. Palun muuda faili nimi pärast allalaadimist nimi sobivaks vastavalt sellele, millise linnu elupaikadele indekseid parasjagu arvutad. Näiteks, "maastikuindeksid_rukkiraak.csv" kui arvutad rukkiräägule.

In [12]:
import pandas as pd
import pylandstats as pls

all_metrics = []

for landscape in tqdm(all_tifs):
    ls = pls.Landscape(landscape)
    metrics_df = ls.compute_landscape_metrics_df(metrics=["edge_density", "patch_density", "shannon_diversity_index"])
    metric_df = metric_df.rename(columns={"edge_density": "ED", "patch_density": "PD", "shannon_diversity_index": "SDI"})
    metrics_df["ED"] = ls.compute_landscape_metrics_df(metrics=["edge_density"])
    metrics_df["PD"] = ls.compute_landscape_metrics_df(metrics=["patch_density"])
    metrics_df["MSI"] = ls.shape_index()['shape_index'].mean()
    metrics_df["MPS"] = ls.area(class_val=None, hectares=True)['area'].mean()
    metrics_df.index = [landscape]
    all_metrics.append(metrics_df)
    
all_metrics_df = pd.concat(all_metrics, axis=0)

all_metrics_df.to_csv("landscape_metrics.csv", sep=",", encoding="utf-8")

display(all_metrics_df)

100%|██████████| 50/50 [00:00<00:00, 57.24it/s]


,edge_density,patch_density,shannon_diversity_index,ED,PD,MSI,MPS
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668
63580_clip.tif,43.288772,3.500567,1.491801,NaN,NaN,1.485714,28.5668


[lae tabel alla siit](landscape_metrics.csv)

